In [1]:
import numpy as np

In [2]:
test_vec = np.random.rand(9)
test_vec

array([0.60258853, 0.01069432, 0.4716336 , 0.49522994, 0.42048393,
       0.69663125, 0.64570479, 0.26826099, 0.12019827])

In [3]:
test_mat = np.zeros((9,9))
test_mat[:,-1] = test_vec
test_mat[-1,:] = test_vec  

In [4]:
a = test_vec[-1]
temp = np.sqrt(a**2 + 4 * np.sum(test_vec[:-1]**2))
l1 = (a + temp) / 2
l2 = (a - temp) / 2
l1, l2

(1.469187706499764, -1.3489894407966767)

In [5]:
v1 = np.array([*(test_vec[:-1] / l1) ,1]) 
v1 = v1 / np.linalg.norm(v1)

v2 = np.array([*(test_vec[:-1] / l2) ,1])
v2 = v2 / np.linalg.norm(v2)

In [6]:
np.allclose(l1 * np.outer(v1, v1) + l2 * np.outer(v2,v2), test_mat)

True

In [8]:
test_mat = np.arange(9).reshape(3,3)
test_mat = test_mat.T @ test_mat

chol1 = np.linalg.cholesky(test_mat)
chol2 = np.linalg.cholesky(test_mat + np.outer([0,0,1], [0,0,1]))

In [15]:
def cholesky_update(chol, update_vector, multiplier=np.array([1.])):
    dtype = np.common_type(chol, update_vector, multiplier)
    chol = np.array(chol, dtype=dtype)
    update_vector = np.array(update_vector, dtype=dtype)
    multiplier = np.array(multiplier, dtype=dtype)

    batch_shape = np.broadcast(chol, update_vector, multiplier).shape[:-2]
    chol = np.broadcast_to(chol, batch_shape + chol.shape[-2:])
    update_vector = np.broadcast_to(update_vector, batch_shape + update_vector.shape[-1:])
    multiplier = np.broadcast_to(multiplier, batch_shape)

    chol_diag = np.diagonal(chol, axis1=-2, axis2=-1)

    def compute_new_column(accumulated_quantities, state):
        _, _, omega, b = accumulated_quantities
        index, diagonal_member, col, col_mask = state
        omega_at_index = omega[..., index]

        new_diagonal_member = np.sqrt(
            np.square(diagonal_member) + multiplier / b * np.square(omega_at_index))
        scaling_factor = (np.square(diagonal_member) * b +
                          multiplier * np.square(omega_at_index))

        omega = omega - (omega_at_index / diagonal_member)[..., np.newaxis] * col
        new_col = new_diagonal_member[..., np.newaxis] * (
            col / diagonal_member[..., np.newaxis] +
            (multiplier * omega_at_index / scaling_factor)[..., np.newaxis] * omega * col_mask)
        b = b + multiplier * np.square(omega_at_index / diagonal_member)
        return new_diagonal_member, new_col, omega, b

    cols_mask = np.tril(np.ones_like(chol[0]))
    chol = np.moveaxis(chol, -1, 0)
    chol_diag = np.moveaxis(chol_diag, -1, 0)

    new_diag, new_chol, _, _ = np.apply_along_axis(
        compute_new_column,
        axis=0,
        arr=(np.arange(chol.shape[0]), chol_diag, chol, cols_mask),
        initializer=(
            np.zeros_like(multiplier),
            np.zeros_like(chol[0]),
            update_vector,
            np.ones_like(multiplier)))
    new_chol = np.moveaxis(new_chol, 0, -1)
    new_diag = np.moveaxis(new_diag, 0, -1)
    new_chol = np.linalg.set_diag(new_chol, new_diag)
    return new_chol

In [82]:
def cholesky_update(chol, update_vector, multiplier=1.):
    omega = update_vector
    b = 1
    
    new_chol = np.zeros_like(chol)
    for i in range(chol.shape[0]):
        new_chol[i,i] = np.sqrt(chol[i,i]**2 + multiplier / b * omega[i]**2)
        scaling_factor = (chol[i,i]**2 * b + multiplier * omega[i]**2)
        for k in range(i+1, chol.shape[0]):
            omega[k] -= omega[i] / chol[i,i] * chol[k, i]
            new_chol[k, i] = new_chol[i,i] / chol[i,i] * chol[k, i] + new_chol[i,i] * multiplier * omega[i] / scaling_factor * omega[k]
        b += multiplier * omega[i]**2 / chol[i,i]**2
    return new_chol

In [83]:
np.allclose(cholesky_update(chol1, np.array([0,0,1], dtype=float)), chol2) 

True

In [84]:
chol2

array([[6.70820393, 0.        , 0.        ],
       [8.04984472, 1.09544512, 0.        ],
       [9.39148551, 2.19089023, 1.        ]])

In [85]:
cholesky_update(chol1, np.array([0,0,1], dtype=float))

array([[6.70820393, 0.        , 0.        ],
       [8.04984472, 1.09544512, 0.        ],
       [9.39148551, 2.19089023, 1.        ]])